In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from parseq.datasets import CFQDatasetLoader
from parseq.scripts_resplit.resplit_cfq import FrequencyDistribution, DivergenceComputer, compute_approx_chernoff_change
import random, json
from typing import List
from parseq.grammar import taglisp_to_tree, tree_to_taglisp
from nltk import Tree
from tqdm import tqdm
from copy import copy

In [60]:
ds = CFQDatasetLoader().load(split="mcd1/modent", keepids=True, loadunused=True)
xs = ds.examples
random.shuffle(xs)

CFQDatasetLoader: make data
CFQDatasetLoader: make data in 0.0 sec
loading split 'mcd1'
splitting off a random 10% of 'train' for 'iidvalid' using seed 42
doing 'train'


100%|██████████| 86169/86169 [00:23<00:00, 3651.51it/s]


doing 'test'


100%|██████████| 11968/11968 [00:03<00:00, 3109.04it/s]


doing 'iidvalid'


100%|██████████| 9574/9574 [00:02<00:00, 3660.36it/s]


doing 'oodvalid'


100%|██████████| 11968/11968 [00:03<00:00, 3049.26it/s]


doing 'unused'


100%|██████████| 119678/119678 [00:38<00:00, 3137.13it/s]


In [17]:
N = 40000
with open(f"minicfq{N}unsplit1.json", "r") as f:
    xs = json.load(f)
random.shuffle(xs)

In [4]:
def print_divergences(subsets, cache=None):
    dc = DivergenceComputer()
    # updating compound distribution and computing divergences
    print("updating compound distributions and computing divergences")
    cds = {}
    ads = {}
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)
        ads[str(i)] = dc.compute_atom_distribution(subset)
    divs = dc._compute_compound_divergences(cds)
    adivs = dc._compute_atom_divergences(ads)
    overlaps = dc._compute_overlaps(cds)
    aoverlaps = dc._compute_overlaps(ads, union=True)
    for k in ["0-1", "0-2", "1-2"]:
        print(f"{k}: {divs[k]:.3f}, {overlaps[k]*100:.1f} -- {adivs[k]:.3f}, {aoverlaps[k]*100:.1f}")
        
        
def iter_putinbest(xs, statses, alpha=0.1):
    for example in tqdm(xs):
        comps = dc.extract_compounds(example[2])
        changes = compute_approx_chernoff_change(comps, *statses, alpha=0.1)
        changes = list(zip(changes, range(3)))
        bestchoices = sorted(changes, key=lambda x: x[0])   # which subset is best
#             print(bestchoices)
        bestchoices = [x[1] for x in bestchoices]

        bestchoice = bestchoices[0]
        if len(subsets[bestchoice]) <= sizes[bestchoice] * n:
            subsets[bestchoice].append(example)
            n += 1
            for comp in comps:
                statses[bestchoice][comp] += 1
            if n >= targetsize:
                done = True
                print("done!")
                break
        else:
            newxs.append(example)
    

        
def make_mcd_splits(xs:List, sizes=(0.6, 0.2, 0.2), backbleed=0.1, minbackbleed=20, restfraq=0.2):
    # initialize randomly
    dc = DivergenceComputer()
    assert sum(sizes) == 1
    sizes = [int(round(size * len(xs) * (1 - restfraq))) for size in sizes]
#     sizes[0] = len(xs) - sum(sizes[1:])
#     assert sum(sizes) == len(xs)
    print("Target sizes: ", sizes)
    
    # build comp cache
    print("building comp cache")
    for example in tqdm(xs):
        comps = dc.extract_compounds(example[2])
        treestr = tree

In [5]:
def print_divergences(subsets, cache=None):
    dc = DivergenceComputer()
    # updating compound distribution and computing divergences
    print("updating compound distributions and computing divergences")
    cds = {}
    ads = {}
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)
        ads[str(i)] = dc.compute_atom_distribution(subset)
    divs = dc._compute_compound_divergences(cds)
    adivs = dc._compute_atom_divergences(ads)
    overlaps = dc._compute_overlaps(cds)
    aoverlaps = dc._compute_overlaps(ads, union=True)
    for k in ["0-1", "0-2", "1-2"]:
        print(f"{k}: {divs[k]:.3f}, {overlaps[k]*100:.1f} -- {adivs[k]:.3f}, {aoverlaps[k]*100:.1f}")
        
        
def iter_putinbest(xs, statses, alpha=0.1):
    for example in tqdm(xs):
        comps = dc.extract_compounds(example[2])
        changes = compute_approx_chernoff_change(comps, *statses, alpha=0.1)
        changes = list(zip(changes, range(3)))
        bestchoices = sorted(changes, key=lambda x: x[0])   # which subset is best
#             print(bestchoices)
        bestchoices = [x[1] for x in bestchoices]

        bestchoice = bestchoices[0]
        if len(subsets[bestchoice]) <= sizes[bestchoice] * n:
            subsets[bestchoice].append(example)
            n += 1
            for comp in comps:
                statses[bestchoice][comp] += 1
            if n >= targetsize:
                done = True
                print("done!")
                break
        else:
            newxs.append(example)
    

        
def make_mcd_splits(xs:List, sizes=(0.6, 0.2, 0.2), backbleed=0.1, minbackbleed=20, restfraq=0.2):
    # initialize randomly
    dc = DivergenceComputer()
    assert sum(sizes) == 1
    sizes = [int(round(size * len(xs) * (1 - restfraq))) for size in sizes]
#     sizes[0] = len(xs) - sum(sizes[1:])
#     assert sum(sizes) == len(xs)
    print("Target sizes: ", sizes)
    
    # build comp cache
    print("building comp cache")
    compcache = dict()
    compdic = dict()
    rcompdic = dict()
    for example in tqdm(xs):
        treestr = taglisp_to_tree(example[2])
        comps = dc.extract_compounds(treestr)
        treestr = tree_to_taglisp(treestr)
        if treestr not in comps:
            compcache[treestr] = set()
        for comp in comps:
            if comp not in compdic:
                compdic[comp] = len(compdic) + 1
                rcompdic[compdic[comp]] = comp
            compcache[treestr].add(compdic[comp])

    print("initializing randomly by selecting 1 random example for train")
    xs = [(x[0], x[1], taglisp_to_tree(x[2]) if not isinstance(x[2], Tree) else x[2]) for x in xs]
    random.shuffle(xs)
    subsets = [[], [], []]
    
    trainseed = xs.pop(0)
    subsets[0].append(trainseed)
    
    # find validseed example with which trainseed has least overlap
    bestscore, bestid = 1e6, None
    trainseeddist = dc.extract_compound_dist(trainseed[2])
    for i, example in tqdm(enumerate(xs)):
        comps = compcache[tree_to_taglisp(example[2])]
#         exampledist = dc.extract_compound_dist(example[2])
        a, _, _, _ = trainseeddist.compute_overlap(comps)   # how many of trainseed's compounds occur in example's compounds
        if a < bestscore:
            bestscore = a
            bestid = i
    validseed = xs.pop(bestid)
    subsets[1].append(validseed)
    print("selected validation seed example")
    
    # find testseed example with which trainseed and validseed have least overlap
    bestscore, bestid = 1e6, None
    validseeddist = dc.extract_compound_dist(validseed[2])
    seeddist = trainseeddist + validseeddist
    for i, example in tqdm(enumerate(xs)):
        comps = compcache[tree_to_taglisp(example[2])]
#         exampledist = dc.extract_compound_dist(example[2])
        a, _, _, _ = seeddist.compute_overlap(comps)
        if a < bestscore:
            bestscore = a
            bestid = i
    testseed = xs.pop(bestid)
    subsets[2].append(testseed)
    print("selected test seed example")
    print("initialized")

    trainstats = FrequencyDistribution()
    validstats = FrequencyDistribution()
    teststats = FrequencyDistribution()
    statses = [trainstats, validstats, teststats]
            

    # update stats
    for subset, stats in zip(subsets, statses):
        for example in subset:
            for comp in compcache[tree_to_taglisp(example[2])]: #dc.extract_compounds(example[2]):
                stats[comp] += 1
    
    while any([len(ss) < s for ss, s in zip(subsets, sizes)]):
        done = False
        while not done:
            newxs = []
    #         print(xs[0], len(xs))

            # insert in best
            for example in tqdm(xs):
                comps = compcache[tree_to_taglisp(example[2])] #dc.extract_compounds(example[2])
                changes = compute_approx_chernoff_change(comps, *statses, alpha=0.1)
                changes = list(zip(changes, range(3)))
                bestchoices = sorted(changes, key=lambda x: x[0])   # which subset is best
        #             print(bestchoices)
                bestchoices = [x[1] for x in bestchoices]

                bestchoice = bestchoices[0]
                if len(subsets[bestchoice]) / sum([len(s) for s in subsets]) <= sizes[bestchoice] / sum(sizes):
                    subsets[bestchoice].append(example)
                    for comp in comps:
                        statses[bestchoice][comp] += 1
                    if all([len(ss) >= s for ss, s in zip(subsets, sizes)]):
                        done = True
                        print("done1!")
                        break
                else:
                    newxs.append(example)

            if not done:
                inserted, mininserted = len(xs) - len(newxs), max(backbleed * len(xs), minbackbleed)

                print(f"Inserted {inserted} of at least {mininserted}")
                if inserted < mininserted:
                    print("backloading")
                    news = []
                    for j, example in tqdm(enumerate(newxs)):
                        if inserted >= mininserted or done:
                            print(f"Carrying over {len(newxs[j:])} last examples")
                            news = news + newxs[j:]
                            print(len(news))
                            break
                        comps = compcache[tree_to_taglisp(example[2])] #dc.extract_compounds(example[2])
                        changes = compute_approx_chernoff_change(comps, *statses, alpha=0.1)
                        changes = list(zip(changes, range(3)))
                        bestchoices = sorted(changes, key=lambda x: x[0])   # which subset is best
            #             print(bestchoices)
                        bestchoices = [x[1] for x in bestchoices]

                        didinsert = False
                        for bestchoice in bestchoices:
                            if len(subsets[bestchoice]) / sum([len(s) for s in subsets]) <= sizes[bestchoice] / sum(sizes):
                                didinsert = True
                                subsets[bestchoice].append(example)
                                inserted += 1
                                for comp in comps:
                                    statses[bestchoice][comp] += 1
                                if all([len(ss) >= s for ss, s in zip(subsets, sizes)]):
                                    done = True
                                    print("done2!")
                                break
                        if not didinsert:
                            news.append(example)
                    print(f"Additionally loaded {len(newxs) - len(news)} examples. {inserted}/{mininserted}")
                    newxs = news

            print(len(subsets[0]), len(subsets[1]), len(subsets[2]))    

            print_divergences(subsets)

            xs = newxs
            random.shuffle(xs)
            print("remaining: ", len(xs))
            if len(xs) == 0:
                done = True

        # iterate over remaining
        # assign to the most fitting subset
        print(len(subsets[0]), len(subsets[1]), len(subsets[2]))    

        print("pruning by atom distributions")
        subsets = prune_byatom(subsets)
        print(len(subsets[0]), len(subsets[1]), len(subsets[2]))
        print_divergences(subsets)

    
    print(len(subsets[0]), len(subsets[1]), len(subsets[2]))    
    assert all([len(ss) == s for ss, s in zip(subsets, sizes)])
        
    return subsets


def prune_byatom(subsets):
    dc = DivergenceComputer()
    trainatoms = set()
    for example in subsets[0]:  # train examples
        atoms = dc.extract_atoms(example[2])
        trainatoms |= set(atoms)
        
    newvalid = []
    for example in subsets[1]:  # validation examples
        atoms = dc.extract_atoms(example[2])
        if len(set(atoms) - trainatoms) > 0:
#             print("example contains OOD atoms")
            pass
        else:
            newvalid.append(example)
            
    newtest = []
    for example in subsets[2]:  # test examples
        atoms = dc.extract_atoms(example[2])
        if len(set(atoms) - trainatoms) > 0:
#             print("example contains OOD atoms")
            pass
        else:
            newtest.append(example)
            
    print(f"found {len(subsets[1]) - len(newvalid)} validation examples using unseen atom")
    print(f"found {len(subsets[2]) - len(newtest)} test examples using unseen atom")
            
    subsets = [subsets[0], newvalid, newtest]
    return subsets

In [21]:
print(len(xs))
# _xs = xs[:5000]
# _xs = xs[:200000]
_xs = xs
dss = make_mcd_splits(_xs, restfraq=0.5)
# print(dss)

40000
Target sizes:  [12000, 4000, 4000]
building comp cache


100%|██████████| 40000/40000 [01:16<00:00, 522.19it/s]


initializing randomly by selecting 1 random example for train


39999it [00:01, 29818.02it/s]


selected validation seed example


...

12000 4000 4000
updating compound distributions and computing divergences


100%|██████████| 4000/4000 [00:00<00:00, 6368.00it/s]


0-1: 0.935, 4.4 -- 0.085, 98.1
0-2: 0.869, 4.4 -- 0.059, 97.1
1-2: 0.817, 15.1 -- 0.042, 99.0
remaining:  4546
12000 4000 4000
pruning by atom distributions
found 0 validation examples using unseen atom
found 0 test examples using unseen atom
12000 4000 4000
updating compound distributions and computing divergences


100%|██████████| 4000/4000 [00:00<00:00, 6181.43it/s]


0-1: 0.935, 4.4 -- 0.085, 98.1
0-2: 0.869, 4.4 -- 0.059, 97.1
1-2: 0.817, 15.1 -- 0.042, 99.0
12000 4000 4000


In [22]:
len(dss[0]), len(dss[1]), len(dss[2])

(12000, 4000, 4000)

In [23]:
# split up in more parts
_dss = [[x[0] for x in xs] for xs in dss]
validsize = len(_dss[1]) // 2
random.shuffle(_dss[0])
trainindexes = _dss[0][:-validsize]
iidvalidindexes = _dss[0][-validsize:]
random.shuffle(_dss[1])
ood2validindexes = _dss[1][-validsize:]
random.shuffle(_dss[2])
testindexes = _dss[2][:-validsize]
oodvalidindexes = _dss[2][-validsize:]
tosave = {
    "train": trainindexes,
    "test": testindexes,
    "iidvalid": iidvalidindexes,
    "oodvalid": oodvalidindexes,
    "ood2valid": ood2validindexes
}

# save
NN = sum([len(x) for k, x in tosave.items()])
print(NN)
with open(f"minicfq{NN}split1.json", "w") as f:
    json.dump(tosave, f)

18000


In [67]:
tosave["train"]

[103264,
 58184,
 220817,
 161060,
 138458,
 23973,
 103775,
 11767,
 194600,
 40685,
 223365,
 132205,
 77021,
 21401,
 202580,
 13435,
 60871,
 168858,
 13709,
 83586,
 77466,
 41286,
 206002,
 92808,
 230524,
 97005,
 110676,
 84661,
 85518,
 210941,
 230233,
 187014,
 34015,
 218415,
 89293,
 194422,
 222868,
 222031,
 96057,
 15631,
 133936,
 140176,
 34477,
 15462,
 12458,
 155916,
 216978,
 34167,
 161814,
 12700,
 56268,
 203792,
 37890,
 133382,
 7656,
 35503,
 135789,
 96913,
 21807,
 153912,
 42789,
 90038,
 35753,
 101906,
 238077,
 156440,
 141253,
 203452,
 126126,
 198207,
 210631,
 154783,
 237133,
 187881,
 109570,
 91447,
 91362,
 156273,
 98152,
 99181,
 20997,
 61262,
 155567,
 84127,
 38012,
 153473,
 227524,
 220081,
 91104,
 22190,
 30082,
 162180,
 234855,
 154406,
 35289,
 30705,
 187663,
 80436,
 102713,
 19194,
 8977,
 229469,
 195759,
 224814,
 48408,
 68028,
 99521,
 231536,
 108984,
 40081,
 67210,
 38128,
 96832,
 196982,
 14774,
 204250,
 50459,
 11517,


In [56]:
random.shuffle(_xs)
rdss = [_xs[:1200], _xs[1200:1600], _xs[1600:2000]]
# rdss = [_xs[:2400], _xs[2400:3200], _xs[3200:4000]]
# rdss = [_xs[:1500], _xs[1500:2000], _xs[2000:2500]]
print_divergences(rdss)

updating compound distributions and computing divergences


100%|██████████| 400/400 [00:00<00:00, 3360.61it/s]


0-1: 0.539, 14.4 -- 0.002, 99.0
0-2: 0.488, 15.9 -- 0.002, 99.0
1-2: 0.665, 23.9 -- 0.002, 100.0


In [ ]:
dc = DivergenceComputer()
totalatomdist = dc.compute_atom_distribution(_xs)
totalatomdist = [(k, v) for k, v in totalatomdist._counts.items()]
totalatomdist = sorted(totalatomdist, key=lambda x: x[1])
print(json.dumps(totalatomdist, indent=4))

In [ ]:
def compute_example_overlap(comps, dist):   # how many of the compounds have already been observed in the given dist
    overlap = 0
    unoverlap = 0
    total = 0
    for comp in comps:
        if comp in dist and dist[comp] > 0:
            overlap += 1
        else:
            unoverlap += 1
        total += 1
    return overlap / total, unoverlap / total


def compute_example_stats(subsets, cds, dc=None):
    print("Computing stats for all examples")
    stats = {}
    cds = {k: v.tofreqs() for k, v in cds.items()}
    for subset in subsets:
        for example in tqdm(subset):
            eid = example[0]
            ecomp = dc.extract_compound_dist(example[2]).tofreqs()
            # print(len(ecomp))
            stats[eid] = (
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.9),
            )
    return stats


def make_mcd_splits2(xs:List, sizes=(0.6, 0.2, 0.2), swapsize=10, iters=10):
    assert len(sizes) in (3,)
    # make random assignments and compute stats
    print("making random assigments")
    xs = [(x[0], x[1], taglisp_to_tree(x[2]) if not isinstance(x[2], Tree) else x[2]) for x in xs]
    random.shuffle(xs)
    subsets = []
    prev = 0
    for i, s in enumerate(sizes):
        if i < len(sizes):
            c = int(round(s * len(xs))) + prev
            subsets.append(xs[prev:c])
            prev = c
        else:
            subsets.append(xs[prev:])

    print("Computing initial compound distributions and their divergences")
    cds = {}
    dc = DivergenceComputer()
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)

    divs = dc._compute_compound_divergences(cds)
    print("Initial compound divergences")
    print(json.dumps(divs, indent=3))

    """
    print(subsets[0][0])
    comps = dc.extract_compounds(taglisp_to_tree(subsets[0][0][2]))
    print(compute_example_overlap(comps, cds["2"]))
    compdist = dc.extract_compound_dist(taglisp_to_tree(subsets[0][0][2]))
    print(FrequencyDistribution.compute_chernoff_coeff(cds["2"], compdist, alpha=0.1))
    """

    print("Iterating")
    # in every iteration, we find for each split the examples that would improve the divergence most and perform one swap
    # remove from train set all compounds that are present in test-ood and ood2
    # remove from ood2 all compounds that are present in test-ood
    # ==> test-ood would contain entirely novel compounds that have not been trained on and have not been validated with
    while iters > 0:
        stats = compute_example_stats(subsets, cds, dc=dc)   # initial stats

        # train --> test-ood
        examples = subsets[0]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintotest, traintotrain = scored[:swapsize], scored[swapsize:]

        # train -> ood2
        examples = traintotrain
        scored = [(i, stats[examples[i][0]][1]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintoood2, traintotrain = scored[:swapsize], scored[swapsize:]

        # ood2 -> test-ood
        examples = subsets[1]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totest, ood2toood2 = scored[:swapsize], scored[swapsize:]

        # ood2 -> train
        examples = ood2toood2
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totrain, ood2toood2 = scored[:swapsize], scored[swapsize:]

        # test-ood --> train
        examples = subsets[2]
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtotrain, testtotest = scored[:swapsize], scored[swapsize:]

        # test-ood --> ood2
        examples = testtotest
        scored = [(i, stats[examples[i][0]][4]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtoood2, testtotest = scored[:swapsize], scored[swapsize:]

        subsets = [traintotrain + ood2totrain + testtotrain,
                   ood2toood2 + traintoood2 + testtoood2,
                   testtotest + traintotest + ood2totest]


        # updating compound distribution and computing divergences
        print("updating compound distributions and computing divergences")
        cds = {}
        for i, subset in enumerate(subsets):
            cds[str(i)] = dc.compute_compound_distribution(subset)
        divs = dc._compute_compound_divergences(cds)
        print(json.dumps(divs, indent=3))

        iters -= 1


    return cds

In [77]:
def compute_example_overlap(comps, dist):   # how many of the compounds have already been observed in the given dist
    overlap = 0
    unoverlap = 0
    total = 0
    for comp in comps:
        if comp in dist and dist[comp] > 0:
            overlap += 1
        else:
            unoverlap += 1
        total += 1
    return overlap / total, unoverlap / total


def compute_example_stats(subsets, cds, dc=None):
    print("Computing stats for all examples")
    stats = {}
    cds = {k: v.tofreqs() for k, v in cds.items()}
    for subset in subsets:
        for example in tqdm(subset):
            eid = example[0]
            ecomp = dc.extract_compound_dist(example[2]).tofreqs()
            # print(len(ecomp))
            stats[eid] = (
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.9),
            )
    return stats


def make_mcd_splits2(xs:List, sizes=(0.6, 0.2, 0.2), swapsize=10, iters=10):
    assert len(sizes) in (3,)
    # make random assignments and compute stats
    print("making random assigments")
    xs = [(x[0], x[1], taglisp_to_tree(x[2]) if not isinstance(x[2], Tree) else x[2]) for x in xs]
    random.shuffle(xs)
    subsets = []
    prev = 0
    for i, s in enumerate(sizes):
        if i < len(sizes):
            c = int(round(s * len(xs))) + prev
            subsets.append(xs[prev:c])
            prev = c
        else:
            subsets.append(xs[prev:])

    print("Computing initial compound distributions and their divergences")
    cds = {}
    dc = DivergenceComputer()
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)

    divs = dc._compute_compound_divergences(cds)
    print("Initial compound divergences")
    print(json.dumps(divs, indent=3))

    """
    print(subsets[0][0])
    comps = dc.extract_compounds(taglisp_to_tree(subsets[0][0][2]))
    print(compute_example_overlap(comps, cds["2"]))
    compdist = dc.extract_compound_dist(taglisp_to_tree(subsets[0][0][2]))
    print(FrequencyDistribution.compute_chernoff_coeff(cds["2"], compdist, alpha=0.1))
    """

    print("Iterating")
    # in every iteration, we find for each split the examples that would improve the divergence most and perform one swap
    # remove from train set all compounds that are present in test-ood and ood2
    # remove from ood2 all compounds that are present in test-ood
    # ==> test-ood would contain entirely novel compounds that have not been trained on and have not been validated with
    while iters > 0:
        stats = compute_example_stats(subsets, cds, dc=dc)   # initial stats

        # train --> test-ood
        examples = subsets[0]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintotest, traintotrain = scored[:swapsize], scored[swapsize:]

        # train -> ood2
        examples = traintotrain
        scored = [(i, stats[examples[i][0]][1]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintoood2, traintotrain = scored[:swapsize], scored[swapsize:]

        # ood2 -> test-ood
        examples = subsets[1]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totest, ood2toood2 = scored[:swapsize], scored[swapsize:]

        # ood2 -> train
        examples = ood2toood2
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totrain, ood2toood2 = scored[:swapsize], scored[swapsize:]

        # test-ood --> train
        examples = subsets[2]
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtotrain, testtotest = scored[:swapsize], scored[swapsize:]

        # test-ood --> ood2
        examples = testtotest
        scored = [(i, stats[examples[i][0]][4]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtoood2, testtotest = scored[:swapsize], scored[swapsize:]

        subsets = [traintotrain + ood2totrain + testtotrain,
                   ood2toood2 + traintoood2 + testtoood2,
                   testtotest + traintotest + ood2totest]


        # updating compound distribution and computing divergences
        print("updating compound distributions and computing divergences")
        cds = {}
        for i, subset in enumerate(subsets):
            cds[str(i)] = dc.compute_compound_distribution(subset)
        divs = dc._compute_compound_divergences(cds)
        print(json.dumps(divs, indent=3))

        iters -= 1


    return cds

In [85]:
dss = make_mcd_splits(xs[:1000])

initializing randomly by selecting 1 random example for each subset


TypeError: list indices must be integers or slices, not tuple

In [86]:
### len(dss[0]), len(dss[1]), len(dss[2])

In [9]:
dss[0]._counts

AttributeError: 'list' object has no attribute '_counts'